# SVM Spam Filter
#### Miloslav Homer, Marek Zpěváček

### Matematika

#### Objective function

Naším cieľom bude minimalizovať objective function:
$$
    J(\alpha) = \frac{1}{m}\sum_{i=1}^m\left[1-y^{(i)}K^{(i)\top}\alpha\right]_+ + \frac{\lambda}{2}\alpha^\top K\alpha,
$$
kde hľadáme $\alpha$, $m$ je počet správ, $\lambda$ je parameter, ktorý volíme na začiatku a $K$ je Gaussovský kernel, tj:
$$
    K(x,z)=\operatorname{exp}\left(-\frac{1}{2\tau^2}\|x-z\|_2^2\right).
$$
Značením $[t]_+$ rozumieme $\max{(t,0)}$.

### Parsing dát

#### Formát vstupu

Na vstupe dostaneme súbor obsahujúci (v tomto poradí):
počet emailov, dĺžku slovníka (tj počet rôznych slov vyskytujúcich sa v týchto emailoch), slovník (oddelené medzerou), zoznam emailov. Prvé číslo je vždy buď 0 alebo 1, indikuje či je daný email spam. Ďalej zoznam čísel ukončených -1, na $i$-tej pozícii sa nachádza číslo $j$, tj $i$-te slovo emailu je $j$-te slovo slovníka.

### Learning fáza

### Testovacia fáza

### Testy a výsledky